In [1]:
import selenium
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
import json
import requests
%matplotlib inline
import pickle

In [2]:
url = 'https://stats.nba.com/stats/boxscoredefensive'

params = {
    'EndPeriod':10,
    'EndRange':28800,
    'GameID':'0021700784',
    'RangeType':0,
    'Season':2017-18,
    'SeasonType':'Regular+Season',
    'StartPeriod':1,
    'StartRange':0
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

resp = requests.get(url=url, params=params, headers=headers)

In [93]:
resp = requests.get(url="https://stats.nba.com/stats/hustlestatsboxscore",
                    params=params, headers=headers)

In [95]:
resp = requests.get(url="https://stats.nba.com/stats/boxscoredefensive",
                    params=params, headers=headers)

In [98]:
get_df_nba_json(resp.json(), rs=0)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,TEAM_NICKNAME,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,...,FG3_PCT,FTM,SFL,DEF_FLS,CFGM,CFGA,CFG_PCT,CFG3M,CFG3A,CFG3_PCT
0,0021700013,1610612745,HOU,Houston,Rockets,2772,Trevor Ariza,F,,32:26,...,1.000,0,0,0,2,2,1.000,0,0,0.0
1,0021700013,1610612745,HOU,Houston,Rockets,201583,Ryan Anderson,F,,25:39,...,0.000,0,0,0,3,7,0.429,0,0,0.0
2,0021700013,1610612745,HOU,Houston,Rockets,203991,Clint Capela,C,,28:20,...,0.000,0,0,0,3,4,0.750,0,0,0.0
3,0021700013,1610612745,HOU,Houston,Rockets,201569,Eric Gordon,G,,34:49,...,0.667,5,1,1,2,3,0.667,0,0,0.0
4,0021700013,1610612745,HOU,Houston,Rockets,201935,James Harden,G,,37:46,...,0.286,0,0,0,0,2,0.000,0,0,0.0
5,0021700013,1610612745,HOU,Houston,Rockets,201601,Luc Mbah a Moute,,,24:03,...,0.000,0,0,0,1,2,0.500,0,1,0.0
6,0021700013,1610612745,HOU,Houston,Rockets,200782,PJ Tucker,,,29:06,...,0.400,3,2,1,1,2,0.500,0,0,0.0
7,0021700013,1610612745,HOU,Houston,Rockets,2403,Nene,,,17:37,...,1.000,0,0,0,2,5,0.400,0,0,0.0
8,0021700013,1610612745,HOU,Houston,Rockets,1627743,Demetrius Jackson,,,10:14,...,0.000,0,0,0,3,5,0.600,0,0,0.0
9,0021700013,1610612745,HOU,Houston,Rockets,204028,Tarik Black,,DNP - Coach's Decision,0:00,...,NaN,0,0,0,0,0,NaN,0,0,NaN


In [18]:
def get_date_place(game_id, year = "2017"):
    #0021700784
    try:
        resp = requests.get(url="https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/" + str(year) + "/scores/gamedetail/"+ str(game_id) +"_gamedetail.json",
                            headers=headers)
        data = resp.json()["g"]["gdte"]
        place = resp.json()["g"]["an"]
    except Exception as e:
        print(e)
        data = np.nan
        place = np.nan
    
    return(data, place)

In [4]:
def get_df_nba_json(resp_json, date_place = False, rs=0):
    dict_resp = resp_json['resultSets'][rs]
    df_resp = pd.DataFrame(dict_resp["rowSet"])
    df_resp.columns = dict_resp["headers"]
    
    if(date_place):
        game_date, game_place = get_date_place(df_resp.GAME_ID.iloc[0])

        df_resp["GAME_DATE"] = np.repeat(game_date, len(df_resp))
        df_resp["GAME_PLACE"] = np.repeat(game_place, len(df_resp))

        teams = df_resp.TEAM_ABBREVIATION.unique()

        df_resp["GAME"] = np.repeat(teams[0] + " @ " + teams[1] + " " + game_date, len(df_resp))
    
    return(df_resp)

In [5]:
def junta_df_tipos(df_tipos, cols_drop = ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 
                                          'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 
                                          'COMMENT', 'MIN', 'MINUTES']):
    resp = df_tipos[0]
    for i in range(1, len(df_tipos)):
        junta = df_tipos[i]
        colunas_repetidas = list(set(junta.columns).intersection(resp.columns))
        
        junta = junta.drop(cols_drop, axis=1, errors="ignore")
        junta.columns = [str(col) + '_' + lista_sites[i]
                         if col in colunas_repetidas else str(col) 
                         for col in junta.columns]
        
        resp = resp.merge(junta, how="left", 
                          left_index=True, right_index=True)
    return(resp)

In [6]:
lista_sites = ["traditional", "advanced", "scoring", "misc", "usage", "fourfactors", "playertrack", "hustle", "defensive"]

In [34]:
game_ids = []
for i in range(1, 1231):
    game_ids.append("002150" + ('{0:0>4}'.format(i)))

year = 2015

In [35]:
params = {
    'EndPeriod':10,
    'EndRange':28800,
    'GameID':'0021600784',
    'RangeType':0,
    'Season':2016-17,
    'SeasonType':'Regular+Season',
    'StartPeriod':1,
    'StartRange':0
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

df_full = []
df_full_jogo = []
erros = []

for game_id in game_ids:
    
    df_tipos = []
    df_tipos_jogo = []
    
    try:
        game_date, game_place = get_date_place(game_id, year)

        for site in lista_sites:
            if(site == "hustle"):
                url = "https://stats.nba.com/stats/hustlestatsboxscore"
                rs = 1

            elif(site == "defensive"):
                url = "https://stats.nba.com/stats/boxscore" + site
                rs = -1

            else:
                url = "https://stats.nba.com/stats/boxscore" + site + "v2"
                rs = 0

            print(game_id + " " + str(game_date) + " - " + site + "              ", end="\r")

            params["GameID"] = game_id

            resp = requests.get(url=url, params=params, headers=headers)
            #time.sleep(0.5)
            
            if(rs == 0):
                df_tipos.append(get_df_nba_json(resp.json(), rs=0))
                df_tipos_jogo.append(get_df_nba_json(resp.json(), rs=1))
            elif(rs == 1):
                df_tipos.append(get_df_nba_json(resp.json(), rs=1))
                df_tipos_jogo.append(get_df_nba_json(resp.json(), rs=2))
            elif(rs == -1):
                df_tipos.append(get_df_nba_json(resp.json(), rs=0))

        df_resp = junta_df_tipos(df_tipos)
        df_resp_jogo = junta_df_tipos(df_tipos_jogo)
        
        df_resp["GAME_DATE"] = np.repeat(game_date, len(df_resp))
        df_resp["GAME_PLACE"] = np.repeat(game_place, len(df_resp))
        
        df_resp_jogo["GAME_DATE"] = np.repeat(game_date, len(df_resp_jogo))
        df_resp_jogo["GAME_PLACE"] = np.repeat(game_place, len(df_resp_jogo))

        teams = df_resp.TEAM_ABBREVIATION.unique()
        game_str = teams[0] + " @ " + teams[1] + " " + game_date
        
        df_resp["GAME"] = np.repeat(game_str, len(df_resp))
        df_resp_jogo["GAME"] = np.repeat(game_str, len(df_resp_jogo))

        df_full.append(df_resp.set_index("GAME"))
        df_full_jogo.append(df_resp_jogo.set_index("GAME"))

        #pickle.dump(df_full, open("df_full.p", "wb"))
        #pickle.dump(df_full_jogo, open("df_full_jogo.p", "wb"))
    except Exception as e:
        erros.append(game_id)
        print(e)
        time.sleep(3)
    

In [40]:
base_nba = pd.concat(df_full)

In [41]:
base_nba_jogo = pd.concat(df_full_jogo)

In [43]:
base_nba_jogo

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,DEFLECTIONS,LOOSE_BALLS_RECOVERED,CHARGES_DRAWN,SCREEN_ASSISTS,BOX_OUTS,PTS_hustle,GAME_DATE,GAME_PLACE
GAME,,,,,,,,,,,,,,,,,,,,,
DET @ ATL 2015-10-27,0021500001,1610612765,Pistons,DET,Detroit,240:00,37,96,0.385,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-10-27,Philips Arena
DET @ ATL 2015-10-27,0021500001,1610612737,Hawks,ATL,Atlanta,240:00,37,82,0.451,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-27,Philips Arena
CLE @ CHI 2015-10-27,0021500002,1610612739,Cavaliers,CLE,Cleveland,240:00,38,94,0.404,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-10-27,United Center
CLE @ CHI 2015-10-27,0021500002,1610612741,Bulls,CHI,Chicago,240:00,37,87,0.425,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-27,United Center
NOP @ GSW 2015-10-27,0021500003,1610612740,Pelicans,NOP,New Orleans,240:00,35,83,0.422,6,...,35.0,28.0,17.0,6.0,3.0,6.0,0.0,95.0,2015-10-27,ORACLE Arena
NOP @ GSW 2015-10-27,0021500003,1610612744,Warriors,GSW,Golden State,240:00,41,96,0.427,9,...,57.0,16.0,15.0,15.0,2.0,10.0,0.0,111.0,2015-10-27,ORACLE Arena
WAS @ ORL 2015-10-28,0021500004,1610612764,Wizards,WAS,Washington,240:00,33,84,0.393,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-10-28,Amway Center
WAS @ ORL 2015-10-28,0021500004,1610612753,Magic,ORL,Orlando,240:00,37,100,0.370,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-28,Amway Center
PHI @ BOS 2015-10-28,0021500005,1610612755,76ers,PHI,Philadelphia,240:00,34,83,0.410,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-10-28,TD Garden


In [38]:
base_nba.to_csv("base_nba_per_player_15_16.csv")

In [39]:
base_nba_jogo.to_csv("base_nba_per_game_15_16.csv")

In [33]:
import gc
gc.collect()

204

In [175]:
print(list(base_nba.columns))

['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'PACE', 'PIE', 'PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT', 'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 'PCT_PTS_FB', 'PCT_PTS_FT', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT', 'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM', 'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT', 'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT', 'BLKA', 'PFD', 'PCT_FGM', 'PCT_FGA', 'PCT_FG3M', 'PCT_FG3A', 'PCT_FTM', 'PCT_FTA', 'PCT_OREB', 'PCT_DREB', 'PCT_REB', 'PCT_AST', 'PCT_TOV', 'PCT_STL', 'PCT_BLK', 'PCT_BLKA', 'PCT_PF', 'PCT_PF

In [176]:
base_nba.to_csv("base_nba_full_17_18.csv")

In [156]:
df_full_junto.to_csv("df_full_junto.csv")

In [167]:
len(list(base_nba.GAME_ID.unique()))

1230

In [170]:
import missingno

In [45]:
gc.disable()

### ANTIGO

In [20]:
#driver = webdriver.Chrome("C:/Users/rafae/Downloads/chromedriver_win32/chromedriver.exe")
driver = webdriver.Ie("C:/Users/rafae/Downloads/IEDriverServer_x64_3.14.0/IEDriverServer.exe")

In [21]:
driver.get("https://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021700784&RangeType=0&Season=2017-18&SeasonType=Regular+Season&StartPeriod=1&StartRange=0")

In [3]:
de_para_siglas = pd.read_csv('de_para_siglas.csv').set_index('nome').to_dict()['sigla']

In [5]:
def trata_df(table_html):
    df = pd.read_html("<table>" + table_html + "</table>")
    df = pd.concat(df)
    return(df)

In [ ]:
#https://stats.nba.com/stats/boxscoreusagev2?EndPeriod=10&EndRange=28800&GameID=0021700784&RangeType=0&Season=2017-18&SeasonType=Regular+Season&StartPeriod=1&StartRange=0
#https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2017/scores/gamedetail/0021700784_gamedetail.json
#https://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021700784&RangeType=0&Season=2017-18&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

In [10]:
#game_pages = ["https://stats.nba.com/game/0021700784/"]
game_pages = []
for i in range(1, 1231):
    game_pages.append("https://stats.nba.com/game/002170" + ('{0:0>4}'.format(i)) + "/")

In [11]:
tipo_dados = ["", "advanced", "scoring", "misc", "usage", "four-factors", "tracking", "hustle", "defense"]

In [12]:
def junta_df_tipos(df_tipos):
    resp = df_tipos[0]
    for i in range(1, len(df_tipos)):
        junta = df_tipos[i]
        colunas_repetidas = list(set(junta.columns).intersection(resp.columns))
        junta = junta.drop(colunas_repetidas, axis=1, errors="ignore")
        
        resp = resp.merge(junta, how="left", 
                          left_index=True, right_index=True)
    return(resp)

In [16]:
driver.implicitly_wait(30)
driver.set_page_load_timeout(100)

In [ ]:
resp = []
erro = []

tent = 0
for g in range(33, len(game_pages)):
    try:
        driver.get(game_pages[g])
        time.sleep(5)
        
        game_date = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'game-summary__date')))

        #game_date = driver.find_element_by_class_name("game-summary__date").get_attribute("innerHTML")
        game_date = game_date.get_attribute("innerHTML")
        game_date = datetime.strptime(game_date, '%b %d, %Y')

        game_str = ""
        team_names_siglas = []

        df_tipos = []

        for tipo in tipo_dados:
            print(str(g) + "... " + game_pages[g] + tipo + "     ", end="\r")
            if(tipo != ""):
                driver.get(game_pages[g] + tipo)
                time.sleep(30)

            teams = driver.find_elements_by_tag_name("nba-stat-table")
            team_names = driver.find_elements_by_class_name("game-summary-team__name")

            df_stats = []

            i = 0
            for team in teams:
                if(tipo == ""):                
                    team_name = team_names[i].find_element_by_tag_name("a").get_attribute("innerHTML")

                    team_name_sigla = team_name
                    for key in de_para_siglas.keys():
                        team_name_sigla = str(team_name_sigla).replace(key, de_para_siglas[key])

                    team_names_siglas.append(team_name_sigla)

                tentativas = 0
                while(tentativas < 3):
                    try:
                        tb_stats = team.find_elements_by_tag_name("table")[0].get_attribute('innerHTML')

                        if tb_stats is not None:
                            break
                    except IndexError:
                        pass
                    time.sleep(2)
                    tentativas += 1

                df_stat_team = trata_df(tb_stats)
                df_stat_team["team"] = np.repeat(team_names_siglas[i], len(df_stat_team))
                df_stat_team["key"] = df_stat_team["Player"] + "_" +  df_stat_team["team"]
                df_stat_team.set_index("key", inplace = True)

                df_stats.append(df_stat_team)
                i += 1 

            df_tipos.append(pd.concat(df_stats))

        df_game = junta_df_tipos(df_tipos)

        game_str = team_names_siglas[0] + " @ " + team_names_siglas[1] + " " + game_date.strftime('%Y-%m-%d')
        
        print(str(g) + ": " + game_pages[g] + tipo + " [" + game_str + "] " + str(len(df_game)))
        
        df_game["game"] = np.repeat(game_str, len(df_game))
        df_game.set_index("game")

        df_game.to_csv("C:/Users/rafae/Google Drive/Projetos/NBA_Games/dfs_nba/"+ str(g) +"_" +game_str+".csv")

        resp.append(df_game)
    except Exception as e:
        print(e)
        if(tent > 3):
            erro.append(game_pages[g])
            try:
                driver.refresh()
            except 
            tent = -1
        else:
            driver.quit()
            driver = webdriver.Ie("C:/Users/rafae/Downloads/IEDriverServer_x64_3.14.0/IEDriverServer.exe")
            driver.implicitly_wait(50)
            driver.set_page_load_timeout(100)
            time.sleep(60)
            g -= 1      
        tent += 1